In [43]:
from __future__ import print_function
import sys
import copy
sys.path.append('../build/')
%pylab inline
np.set_printoptions(precision=4, suppress=True)
import cPickle as pickle
import cv2
import math3d as m3d
import versor as vsr
from motor_estimation import MotorEstimationSolver
from hand_eye_calibration import HandEyeCalibrator
from chessboard import calibrate_lens, find_corners, get_object_pose

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


### Object points

In [44]:
pattern_size = (10,6)
square_size = 0.02
pattern_points = np.zeros( (np.prod(pattern_size), 3), np.float64)
pattern_points[:,:2] = np.indices(pattern_size).T.reshape(-1,2)
pattern_points *= square_size

### Robot poses

In [45]:
rob_pose_list = pickle.load(open('pose_list.dump'))
rob_motors = [vsr.Vec(*pos).trs() * vsr.Rot(vsr.Vec(*axis).unduale() * angle / 2.)
              for pos, (axis, angle) in [(m3d.Transform(pose).pos, 
                                          m3d.Transform(pose).orient.axis_angle)
                                         for pose in rob_pose_list]]

### Object poses

In [46]:
obj_pose_list = pickle.load(open('obj_pose_list.dump'))
obj_motors = [vsr.Vec(*pos).trs() * vsr.Rot(vsr.Vec(*axis).unduale() * angle / 2.)
              for pos, (axis, angle) in [(m3d.Transform(pose).pos, 
                                          m3d.Transform(pose).orient.axis_angle)
                                         for pose in obj_pose_list]]

### Pixels

In [5]:
images = pickle.load(open('image_list.dump'))
camera_matrix, dist_coeffs = calibrate_lens(images)

In [6]:
intrinsics = [camera_matrix[0,0],  camera_matrix[1,1],
              camera_matrix[0,2], camera_matrix[1,2]] + \
              dist_coeffs.tolist()
print(len(intrinsics))

9


In [47]:
corners = [corners for corners in 
           [find_corners(image, pattern_size)[1].reshape(-1,2) 
            for image in images]]

### Initital estimate

In [58]:
n_lines = len(rob_motors)
lines_a = []
lines_b = []
for i in range(n_lines):
    for j in range(i+1,n_lines):
        lines_a.append(((rob_motors[j].rev() * rob_motors[i]).log() * 0.5).unit())
        lines_b.append(((obj_motors[j].rev() * obj_motors[i]).log() * 0.5).unit())
initial_motor = vsr.Mot(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
mes = MotorEstimationSolver(initial_motor)
for a, b in zip(lines_a, lines_b):
    mes.add_line_correspondences_residual_block(b,a)
mes.set_parameterization('BIVECTOR_GENERATOR')
mes.linear_solver_type = "DENSE_QR"
mes.max_num_iterations = 25
mes.function_tolerance = 1e-12
estimated_hand_eye_motor, summary = mes.solve()
print(estimated_hand_eye_motor)
bivectors = [np.array((r * estimated_hand_eye_motor * o.rev()).log()) 
             for r, o in zip(rob_motors, obj_motors)]
estimated_camera_robot_motor = vsr.Dll(*np.average(bivectors,0)).exp()
print(estimated_camera_robot_motor)
print(summary['full_report'])

Mot: [ 0.004 -0.0055 -0.68 -0.74 -0.044 -0.047 0.053 -0.0072 ]
Mot: [ -0.66 -0.26 -0.26 -0.66 -0.3 -0.66 -0.15 -0.1 ]

Solver Summary (v 1.12.0-eigen-(3.2.7)-lapack-suitesparse-(4.4.4)-cxsparse-(3.1.4)-openmp)

                                     Original                  Reduced
Parameter blocks                            1                        1
Parameters                                  8                        8
Effective parameters                        6                        6
Residual blocks                           210                      210
Residual                                 1260                     1260

Minimizer                        TRUST_REGION

Dense linear algebra library            EIGEN
Trust region strategy     LEVENBERG_MARQUARDT

                                        Given                     Used
Linear solver                        DENSE_QR                 DENSE_QR
Threads                                     1                        1
Linear so

In [59]:
for i in range(len(obj_motors)):
    A = rob_motors[i]
    X = estimated_hand_eye_motor
    Z = estimated_camera_robot_motor
    B = obj_motors[i]
    p1 = vsr.Vec(*pattern_points[40]).null().spin(Z.rev() * A * X)
    print(p1)
    p2 = vsr.Vec(*pattern_points[40]).null().spin(B)
    print(p2)
    print()

Pnt: [ 0.04204 0.1157 0.9433 1 0.4525 ]
Pnt: [ 0.02951 0.1021 0.9507 1 0.4576 ]

Pnt: [ 0.046 0.1775 0.928 1 0.4474 ]
Pnt: [ 0.04638 0.171 0.9286 1 0.4469 ]

Pnt: [ -0.1384 0.1527 0.9388 1 0.462 ]
Pnt: [ -0.1383 0.1525 0.938 1 0.4611 ]

Pnt: [ -0.1465 0.1062 0.9426 1 0.4607 ]
Pnt: [ -0.1561 0.09907 0.9457 1 0.4643 ]

Pnt: [ -0.3186 0.08387 0.905 1 0.4638 ]
Pnt: [ -0.3217 0.08077 0.9049 1 0.4645 ]

Pnt: [ -0.3181 0.1411 0.9031 1 0.4683 ]
Pnt: [ -0.31 0.1466 0.9027 1 0.4662 ]

Pnt: [ -0.2504 -0.04202 0.916 1 0.4517 ]
Pnt: [ -0.2429 -0.03701 0.9139 1 0.4478 ]

Pnt: [ -0.1999 -0.07582 0.9242 1 0.4499 ]
Pnt: [ -0.1997 -0.07914 0.9206 1 0.4468 ]

Pnt: [ -0.008437 -0.04911 0.9757 1 0.4773 ]
Pnt: [ -0.01473 -0.05702 0.9749 1 0.4769 ]

Pnt: [ 0.1749 0.06682 1.041 1 0.5598 ]
Pnt: [ 0.1698 0.07036 1.045 1 0.5629 ]

Pnt: [ 0.1472 0.1037 1.032 1 0.5491 ]
Pnt: [ 0.1525 0.1105 1.032 1 0.5505 ]

Pnt: [ 0.002844 0.1123 0.6191 1 0.1979 ]
Pnt: [ 0.002679 0.1064 0.6169 1 0.196 ]

Pnt: [ -0.003107 0.05831 

In [60]:
# A = vsr.Vec(0,0,1).trs() * vsr.Biv(0,0,0).exp()
# X = vsr.Vec(0.1,0, 0).trs() * vsr.Biv(0,0,0).exp()
# Z = vsr.Vec(2, 0, 1).trs() *  vsr.Rot(vsr.Biv(0,1,0) * pi/4)
# B = Z.rev() * A * X
# print(vsr.Vec(0,0,0).null().spin(A * X))
# print(vsr.Vec(0,0,0).null().spin(Z * B))

In [61]:
A.matrix()

array([[-0.04  , -0.7854, -0.6177,  0.0674],
       [ 0.2279,  0.5947, -0.7709,  0.4222],
       [ 0.9729, -0.1717,  0.1552,  0.7171],
       [ 0.    ,  0.    ,  0.    ,  0.    ]])

In [62]:
X.matrix()

array([[ 0.0835, -0.9965,  0.0027,  0.0611],
       [-0.9964, -0.0835, -0.0134,  0.0887],
       [ 0.0136, -0.0015, -0.9999,  0.128 ],
       [ 0.    ,  0.    ,  0.    ,  0.    ]])

In [63]:
p1 = vsr.Vec(*pattern_points[0]).null().spin(Z * B)
print(p1)

Pnt: [ -0.07896 0.3846 0.7762 1 0.3783 ]


In [64]:
p2 = vsr.Vec(*pattern_points[0]).null().spin(A * X)
print(p2)

Pnt: [ -0.08379 0.3902 0.7811 1 0.3847 ]


In [65]:
np.linalg.norm(np.array([-0.2609, -0.2065, 0.7425]) - 
               np.array([-0.2568, -0.2212, 0.7287]))

0.020575227823769093

In [66]:
h = np.array(p2, copy=True)

In [67]:
j = np.array(p1,copy=True)

In [68]:
np.array([-1,-2,-3]) - np.array([-0.5, -1.5, -2.5])

array([-0.5, -0.5, -0.5])

In [82]:
p = vsr.Vec(*pattern_points[25]).null()
print(p)
p = p.spin(X)
print(p)
p = p.spin(A)
print(p)
p = p.spin(Z.rev())
print(p)
p = vsr.Vec(*pattern_points[25]).null().spin(Z.rev() * A * X)
print(p)

Pnt: [ 0.1 0.04 0 1 0.0058 ]
Pnt: [ 0.02955 -0.01426 0.1293 1 0.008899 ]
Pnt: [ -0.002453 0.3207 0.7683 1 0.3466 ]
Pnt: [ -0.1078 0.007956 1.518 1 1.158 ]
Pnt: [ -0.1078 0.007956 1.518 1 1.158 ]


In [83]:
# p = vsr.Vec(*pattern_points[25]).null()
p = vsr.Vec(0,0,0).null()
print(p)
# p = p.spin(Z.rev())
# print(p)
p = p.spin(A)
print(p)
p = p.spin(X)
print(p)
p = vsr.Vec(*pattern_points[25]).null().spin(Z.rev() * A * X)
print(p)

Pnt: [ 0 0 0 1 0 ]
Pnt: [ 0.06741 0.4222 0.7171 1 0.3485 ]
Pnt: [ -0.3521 -0.02334 -0.5887 1 0.2356 ]
Pnt: [ -0.1078 0.007956 1.518 1 1.158 ]


In [84]:
mot = Z.rev() *  A * X
tvec = mot.matrix()[:3,3]
rvec, _ = cv2.Rodrigues(mot.matrix()[:3,:3])
points0 = cv2.projectPoints(pattern_points, rvec, tvec, 
                            camera_matrix, dist_coeffs)[0].reshape(-1,2).astype(np.float32)
mot = B
tvec = mot.matrix()[:3,3]
rvec, _ = cv2.Rodrigues(mot.matrix()[:3,:3])
points1 = cv2.projectPoints(pattern_points, rvec, tvec, camera_matrix,
                            dist_coeffs)[0].reshape(-1,2).astype(np.float32)

bivectors = [np.array((r * X * o.rev()).log()) 
             for r, o in zip(rob_motors, obj_motors)]
Z = vsr.Dll(*np.average(bivectors,0)).exp()
mot = Z.rev() *  A * X
tvec = mot.matrix()[:3,3]
rvec, _ = cv2.Rodrigues(mot.matrix()[:3,:3])
points2 = cv2.projectPoints(pattern_points, rvec, tvec, 
                            camera_matrix, dist_coeffs)[0].reshape(-1,2).astype(np.float32)

In [85]:
p = vsr.Vec(*pattern_points[0]).null().spin(Z.rev() *  A * X)
# p = vsr.Vec(*pattern_points[0]).null().spin(B)

print(p)

focal_length_x = camera_matrix[0,0] 
focal_length_y = camera_matrix[1,1] 
principal_point_x = camera_matrix[0,2]
principal_point_y = camera_matrix[1,2]
k1,k2,p1,p2,k3 = dist_coeffs

x = p[0] / p[2]
y = p[1] / p[2]
r2 = x * x + y * y
r4 = r2 * r2
r6 = r4 * r2
r_coeff = (1 + k1 * r2 + k2 * r4 + k3 * r6)
xd = x * r_coeff + 2 * p1 * x * y + p2 * (r2 + 2 * x * x)
yd = y * r_coeff + 2 * p2 * x * y + p1 * (r2 + 2 * y * y)

# Apply focal length and principal point to get the final image
# coordinates.
image_x = focal_length_x * xd + principal_point_x;
image_y = focal_length_y * yd + principal_point_y;

print(image_x)
print(image_y)

Pnt: [ -0.2146 -0.005152 1.514 1 1.168 ]
340.371134859
364.062676105


In [86]:
points1[0]

array([ 346.5833,  368.3609], dtype=float32)

In [87]:
tuple(corners[0][0])

(615.12787, 409.75198)

In [88]:
cv2.circle(img, tuple(corners[0][0]), 2,(255,0,0), -1)

In [89]:
print(corners[0].shape)
points0 = points0.reshape(-1,2)
points0[0]
xpt = tuple(points0[0])

(60, 2)


In [90]:
repr_corners = repr_corners0[0].reshape(-1,2).astype(np.float32)

NameError: name 'repr_corners0' is not defined

In [91]:
cv2.solvePnP(pattern_points, corners[0], camera_matrix, dist_coeffs)
obj_motors[0].matrix()

array([[ 0.8494, -0.4184,  0.3216,  0.063 ],
       [ 0.3637,  0.9057,  0.2178,  0.0296],
       [-0.3824, -0.068 ,  0.9215,  0.9562],
       [ 0.    ,  0.    ,  0.    ,  0.    ]])

In [79]:
img = images[0].copy()
img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
[cv2.circle(img, tuple(c),    1,(255,0,0), -1) for c in corners[0]]
[cv2.circle(img, tuple(c),    1,(0,255,0), -1) for c in points0]
[cv2.circle(img, tuple(c),    1,(0,0,255), -1) for c in points2]
plt.imshow(img[450:600,500:700], cmap='brg')

NameError: name 'points2' is not defined

In [33]:
from tsai_lens_calibration import TsaiLenzCalibrator
pose_pairs = [(m3d.Transform(A), m3d.Transform(B)) 
              for (A, B)  in zip(rob_pose_list, obj_pose_list)]
tsai_lens = TsaiLenzCalibrator(pose_pairs)
sif = tsai_lens.sensor_in_flange
print(sif)

<Transform:
<Orientation: 
array([[ 0.1246, -0.992 , -0.0178],
       [-0.9841, -0.1213, -0.1294],
       [ 0.1263,  0.0336, -0.9914]])>
<Vector: (0.02149, 0.07718, 0.14263)>
>


In [32]:
tsai_lenz_motor = vsr.Vec(*sif.pos).trs() * \
    vsr.Rot(vsr.Vec(*sif.orient.axis_angle[0]).unduale() * 
            sif.orient.axis_angle[1] / 2.)
X = tsai_lenz_motor

NameError: name 'sif' is not defined

In [55]:
from park_martin import calibrate as park_martin_calibration
A, B = [], []
n_trfs = len(rob_pose_list)
for i in range(n_trfs):
    for j in range(i+1, n_trfs):
        A.append((m3d.Transform(rob_pose_list[j]).inverse * m3d.Transform(rob_pose_list[i])).array)
        B.append((m3d.Transform(obj_pose_list[j]).inverse * m3d.Transform(obj_pose_list[i])).array)
park_martin_calibration(A,B)

(array([[ 0.1051, -0.9945,  0.0021],
        [-0.9943, -0.1051, -0.0175],
        [ 0.0176, -0.0003, -0.9998]]), array([ 0.0412,  0.0991,  0.1449]))

In [ ]:
Z.matrix()

In [ ]:
X.matrix()

In [12]:
def to_array(self): 
    arr = np.zeros((4,4),dtype=np.float64)
    arr[:3,0] = np.array(vsr.Drv(1,0,0).spin(self))
    arr[:3,1] = np.array(vsr.Drv(0,1,0).spin(self))
    arr[:3,2] = np.array(vsr.Drv(0,0,1).spin(self))
    arr[:3,3] = np.array(vsr.Vec(0,0,0).null().spin(self))[:3]
    return arr

In [13]:
vsr.Mot.matrix = to_array

In [ ]:
(vsr.Vec(1,2,3).trs() * vsr.Biv(-pi/4,0,0).exp()).matrix()

In [ ]:
R = vsr.Rot(cos(pi/4), -sin(pi/4), 0, 0)
mot = vsr.Vec(1,2,3).trs() * R
print(mot.matrix())

In [ ]:
R = vsr.Biv(pi/4,0,0).exp()
mot = vsr.Vec(1,2,3).trs() * R
print(mot.matrix())

### Hand Eye Calibration

In [ ]:
print(estimated_hand_eye_motor.matrix())
print(estimated_camera_robot_motor.matrix())
# print(np.array(intrinsics))

In [ ]:
hec = HandEyeCalibrator(estimated_hand_eye_motor, estimated_camera_robot_motor, intrinsics)
print(estimated_hand_eye_motor)
print(estimated_camera_robot_motor)
for a, b in zip(rob_motors, obj_motors):
    for p in pattern_points:
        hec.add_residual_block_2(a, b, vsr.Vec(*p).null())
hec.linear_solver_type = "DENSE_QR"
hec.max_num_iterations = 100
hec.num_linear_solve_threads = 1000
hec.num_threads = 1000
hec.set_parameterization("BIVECTOR_GENERATOR")
# he, cr, intr, summ = hec.solve()
# print(he)
# print(cr)
# print(intr)
# print(summ['full_report'])

In [ ]:
hec = HandEyeCalibrator(estimated_hand_eye_motor, estimated_camera_robot_motor, intrinsics)
for i in range(len(rob_motors)):
    for j in range(len(pattern_points)):
        hec.add_residual_block(rob_motors[i], 
                               vsr.Vec(*pattern_points[j]).null(), 
                               vsr.Vec(corners[i][j][0],corners[i][j][1],0.0))
hec.linear_solver_type = "DENSE_QR"
hec.max_num_iterations = 50
hec.parameter_tolerance = 1e-16
# hec.function_tolerance = 1e-16
hec.parameter_tolerance = 1e-16
hec.num_linear_solve_threads = 1000
hec.num_threads = 1000
hec.set_parameterization("BIVECTOR_GENERATOR")
he, cr, intr, summ = hec.solve()
print(he)
print(cr)
print(intr)
print(summ['full_report'])

In [ ]:
print(vsr.Drv(1,0,0).spin(he))
print(vsr.Drv(0,1,0).spin(he))
print(vsr.Drv(0,0,1).spin(he))
print(vsr.Vec(0,0,0).null().spin(he))

In [ ]:
print(vsr.Drv(1,0,0).spin(cr))
print(vsr.Drv(0,1,0).spin(cr))
print(vsr.Drv(0,0,1).spin(cr))
print(vsr.Vec(0,0,0).null().spin(cr))